##Program 2/2, Data Analysis

Refeeding Data Analysis Script To Create Analytic File For
Doctoral Research Project

Programmer: James Green,
Email: jg1984@shp.rutgers.edu,
Updated: 10/17/2024

Step 1) Load the Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Step 2) New function to import and join files

In [4]:
import os
import pandas as pd

def join_csv(file_name, base_df=None, join_vars=None, join_type='inner', nrows=None):
    """
    Joins a CSV file with an existing DataFrame, or returns the first table if no base_df is provided.

    Parameters:
    - file_name (str): The path to the CSV file to join.
    - base_df (pd.DataFrame or None): The DataFrame to join to. If None, the function returns the CSV as the base DataFrame.
    - join_vars (list or str, optional): Columns to join on (must exist in both DataFrames). Default is None.
    - join_type (str, optional): Type of join - 'left', 'right', 'inner', or 'outer'. Default is 'inner'.
    - nrows (int, optional): Number of rows to load from the CSV. Default is None (loads full table).

    Returns:
    - pd.DataFrame: The result of the join operation or the loaded CSV if no base DataFrame is given.
    """
    try:
        # Load the CSV file with optional row limit
        new_df = pd.read_csv(file_name, nrows=nrows)

        # If no base DataFrame is provided, return the new DataFrame as the starting point
        if base_df is None:
            print(f"Loaded initial DataFrame from {file_name} with {nrows or 'all'} rows.")
            return new_df

        # Ensure join variables are provided before attempting the join
        if join_vars is None:
            raise ValueError("Join variables must be specified for joining.")

        # Perform the join operation
        result_df = pd.merge(base_df, new_df, on=join_vars, how=join_type)

        print(f"Joined {file_name} using {join_type} join on {join_vars} with {nrows or 'all'} rows.")
        return result_df

    except FileNotFoundError:
        print(f"File {file_name} not found.")
        return base_df if base_df is not None else pd.DataFrame()
    except Exception as e:
        print(f"An error occurred: {e}")
        return base_df if base_df is not None else pd.DataFrame()


Step 3) Call the join_csv function to import and join each file to the larger analytic dataframe

In [5]:
# Directory path containing the CSV files
path = '/content/drive/MyDrive/Research/Refeeding/icu/'

# List of tables with join variables, types, and row limits (if needed)
join_instructions = [
    # (file_name, join_columns, join_type, nrows)
    ('icustays.csv', ['subject_id', 'hadm_id', 'stay_id'], None, 100),  # First table: Load only 100 rows
    ('chartevents.csv', ['subject_id', 'hadm_id', 'stay_id'], 'left', 200),  # Load 200 rows for join
    ('inputevents.csv', ['subject_id', 'hadm_id', 'stay_id'], 'left', 50)  # Load 50 rows for join
]

# Initialize base DataFrame as None
base_df = None

# Loop through the join instructions and perform each join sequentially
for file_info in join_instructions:
    file_name, join_vars, join_type, nrows = file_info

    # Construct full file path
    full_path = os.path.join(path, file_name)

    # Call the join function, passing None as base_df for the first file
    base_df = join_csv(
        file_name=full_path,
        base_df=base_df,
        join_vars=join_vars,
        join_type=join_type if join_type else 'inner',
        nrows=nrows
    )

# Save the final DataFrame to a CSV file
output_path = '/content/drive/MyDrive/Research/Refeeding/icu/final_joined_data.csv'
base_df.to_csv(output_path, index=False)

print(f"Final joined DataFrame saved to {output_path}")

# Optionally, print the first few rows to verify
print(base_df.head())


Loaded initial DataFrame from /content/drive/MyDrive/Research/Refeeding/icu/icustays.csv with 100 rows.
Joined /content/drive/MyDrive/Research/Refeeding/icu/chartevents.csv using left join on ['subject_id', 'hadm_id', 'stay_id'] with 200 rows.
Joined /content/drive/MyDrive/Research/Refeeding/icu/inputevents.csv using left join on ['subject_id', 'hadm_id', 'stay_id'] with 50 rows.
Final joined DataFrame saved to /content/drive/MyDrive/Research/Refeeding/icu/final_joined_data.csv
   subject_id   hadm_id   stay_id                      first_careunit  \
0    10000032  29079034  39553978  Medical Intensive Care Unit (MICU)   
1    10000032  29079034  39553978  Medical Intensive Care Unit (MICU)   
2    10000032  29079034  39553978  Medical Intensive Care Unit (MICU)   
3    10000032  29079034  39553978  Medical Intensive Care Unit (MICU)   
4    10000032  29079034  39553978  Medical Intensive Care Unit (MICU)   

                        last_careunit               intime  \
0  Medical Inten